In [2]:
import gzip
import time
import torch
import os
from utils import *
from train_test import *
import argparse
from parse_config import ConfigParser

In [3]:
# Config Loading
parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

In [4]:
data = load_pretrained_data_mind(config)

In [5]:
if config['trainer']['training_type'] == "single_task":
    single_task_training(config, data)
else:
    multi_task_training(config, data)

model training
Training epoch 0/6 - 0.0
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(67.9950, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6016769477529045
valid_socre
valid_scores
early_stopping
Saving checkpoint: out/saved/models/KRED/1224_162557/checkpoint-model-epoch1.pth ...
epoch % self.save_period
Training epoch 1/6 - 0.16666666666666666
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(67.5678, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6122532964868165
valid_socre
valid_scores
early_stopping
Saving checkpoint: out/saved/models/KRED/1224_162557/checkpoint-model-epoch2.pth ...
epoch % self.save_period
Training epoch 2/6 - 0.3333333333333333
######
 Step: 0, 0.0 
######
######
 Step: 100, 0.7352941176470589 
######
all loss: tensor(66.3738, device='cuda:0', grad_fn=<AddBackward0>)
_train_epoch
auc socre: 0.6163112281855666
valid_socre
valid_scores

In [6]:
test_data = data[-1]
testing(test_data, config)

auc score:0.6136670800353788
ndcg score:0.32852655509261247


In [ ]:
if config['trainer']['task'] == "user2item":
    task_index = 0
elif config['trainer']['task'] == "item2item":
    task_index = 4
elif config['trainer']['task'] == "vert_classify":
    task_index = 1
elif config['trainer']['task'] == "pop_predict":
    task_index = 3
model = torch.load('./out/saved/models/KRED/checkpoint.pt')
model.eval()
y_pred = []
start_list = list(range(0, len(test_data['label']), config['data_loader']['batch_size']))
for start in start_list:
    if start + config['data_loader']['batch_size'] <= len(test_data['label']):
        end = start + config['data_loader']['batch_size']
    else:
        end = len(test_data['label'])
    out = model(test_data['item1'][start:end], test_data['item2'][start:end], config['data_loader']['batch_size'])[
        task_index].cpu().data.numpy()

    y_pred.extend(out)
truth = test_data['label']
score = evaluate(y_pred, truth, test_data, config['trainer']['task'])

In [ ]:
user_st = 'U7395_dev'
for user, item, t,p in zip(test_data['item1'], test_data['item2'], truth, y_pred):
    if user != user_st:
        break
    print(user,item, t, p)
